# Initial Coin Offering (ICO) Analysis Notebook

This notebook analyzes Solana ICOs and token launches for potential risks, scams, and anomalies using SolanaGuard's analytical tools.

## Overview

Initial Coin Offerings (ICOs) and token launches on Solana present both opportunities and risks. This analysis aims to identify:

1. **Token distribution patterns** - How widely distributed is the token at launch?
2. **Team wallet behavior** - Are the project founders/team acting responsibly?
3. **Launch metrics** - Analysis of liquidity, pricing, and market activity
4. **Post-launch activity** - Monitoring for suspicious behavior after token launch
5. **Risk indicators** - Identifying common patterns in fraudulent ICOs

In [ ]:
# Import required libraries
import os
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import requests

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import project config
import config

# Import SolanaGuard data collectors
from data_collection.collectors.helius_collector import HeliusCollector
from data_collection.collectors.rugcheck_collector import RugCheckCollector
from data_collection.collectors.vybe_collector import VybeCollector

# Configure plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Create output directories if they don't exist
os.makedirs(config.REPORTS_DIR, exist_ok=True)

In [13]:
# Utility functions for ICO analysis

def calculate_gini_coefficient(values):
    """
    Calculate Gini coefficient (measure of inequality)
    0 = perfect equality, 1 = perfect inequality
    
    Args:
        values: Array of values (e.g., token amounts)
        
    Returns:
        Gini coefficient (float between 0 and 1)
    """
    # Sort values in ascending order
    values = np.asarray(values)
    values = values[np.isfinite(values)]
    if len(values) <= 1 or np.sum(values) == 0:
        return 0
    
    # Sort and cumulate values
    values = np.sort(values)
    index = np.arange(1, len(values) + 1)
    n = len(values)
    
    # Calculate Gini coefficient
    return ((2 * np.sum(np.multiply(index, values))) / (n * np.sum(values))) - ((n + 1) / n)


def calculate_ico_metrics(price_history, liquidity_data):
    """
    Calculate key metrics for token launches based on price and liquidity history
    
    Args:
        price_history: List of price data points (OHLCV)
        liquidity_data: List of liquidity data points
        
    Returns:
        Dictionary of ICO metrics
    """
    metrics = {}
    
    # Price metrics
    if price_history:
        # Extract price data
        try:
            # Initial price (first data point)
            metrics["initial_price"] = price_history[0]["close"] if "close" in price_history[0] else 0
            
            # Peak price
            prices = [p["close"] for p in price_history if "close" in p]
            metrics["price_peak"] = max(prices) if prices else 0
            
            # 7-day price (last data point)
            metrics["price_7d"] = price_history[-1]["close"] if "close" in price_history[-1] else 0
            
            # Price change percentage
            if metrics["initial_price"] > 0:
                metrics["price_change_pct"] = ((metrics["price_7d"] - metrics["initial_price"]) / metrics["initial_price"]) * 100
            else:
                metrics["price_change_pct"] = 0
            
            # Price volatility
            if len(prices) > 1:
                price_changes = [prices[i+1]/prices[i] - 1 for i in range(len(prices)-1) if prices[i] > 0]
                metrics["price_volatility"] = np.std(price_changes) * 100 if price_changes else 0
            else:
                metrics["price_volatility"] = 0
        except Exception as e:
            print(f"Error calculating price metrics: {e}")
    
    # Liquidity metrics
    if liquidity_data:
        try:
            # Initial liquidity
            metrics["initial_liquidity"] = liquidity_data[0]["liquidity_usd"] if isinstance(liquidity_data[0], dict) and "liquidity_usd" in liquidity_data[0] else 0
            
            # Final liquidity
            metrics["final_liquidity"] = liquidity_data[-1]["liquidity_usd"] if isinstance(liquidity_data[-1], dict) and "liquidity_usd" in liquidity_data[-1] else 0
            
            # Liquidity change percentage
            if metrics["initial_liquidity"] > 0:
                metrics["liquidity_change_pct"] = ((metrics["final_liquidity"] - metrics["initial_liquidity"]) / metrics["initial_liquidity"]) * 100
            else:
                metrics["liquidity_change_pct"] = 0
        except Exception as e:
            print(f"Error calculating liquidity metrics: {e}")
    else:
        # If no liquidity data provided, use reasonable defaults
        metrics["initial_liquidity"] = 0
        metrics["final_liquidity"] = 0
        metrics["liquidity_change_pct"] = 0
    
    return metrics

## Initialize API Collectors

First, we initialize the necessary API collectors to gather data from various sources:

In [14]:
# Initialize collectors
helius = HeliusCollector()
vybe = VybeCollector()
rugcheck = RugCheckCollector()

# Verify API connectivity
try:
    helius_status = helius.check_connection()
    rugcheck_status = rugcheck.check_connection()
    vybe_status = vybe.check_connection()
    print(f"Helius API: {'Connected' if helius_status else 'Connection failed'}")
    print(f"RugCheck API: {'Connected' if rugcheck_status else 'Connection failed'}")
    print(f"Vybe API: {'Connected' if vybe_status else 'Connection failed'}")
except Exception as e:
    print(f"Error connecting to APIs: {e}")
    print("Continuing with analysis, but some features may not work properly")

2025-04-25 00:45:26,588 - helius_collector - INFO - Initialized Helius collector
2025-04-25 00:45:26,594 - vybe_collector - INFO - Initialized Vybe collector
2025-04-25 00:45:26,595 - rugcheck_collector - INFO - Initialized RugCheck collector
2025-04-25 00:45:26,594 - vybe_collector - INFO - Initialized Vybe collector
2025-04-25 00:45:26,595 - rugcheck_collector - INFO - Initialized RugCheck collector


Error connecting to APIs: 'HeliusCollector' object has no attribute 'check_connection'
Continuing with analysis, but some features may not work properly


## Identify Recent ICOs and Token Launches

Let's identify recent token launches on Solana for analysis:

In [15]:
# Function to fetch recent token launches
def get_recent_token_launches(days_ago=30, min_liquidity=1000, max_results=10):
    # Define time range
    end_time = int(datetime.now().timestamp())
    start_time = int((datetime.now() - timedelta(days=days_ago)).timestamp())
    
    print(f"Searching for token launches in the last {days_ago} days...")
    
    # Get token launches from RugCheck
    try:
        launches = rugcheck.get_recent_token_launches(
            start_time=start_time,
            end_time=end_time,
            min_liquidity_usd=min_liquidity
        )
        print(f"Found {len(launches)} token launches with min liquidity ${min_liquidity}")
    except Exception as e:
        print(f"Error fetching token launches: {e}")
        launches = []
    
    # Filter and sort by launch date (most recent first)
    if launches:
        launches = sorted(launches, key=lambda x: x.get('launch_timestamp', 0), reverse=True)
        if max_results and max_results < len(launches):
            launches = launches[:max_results]
    
    return launches

# Get recent launches
recent_launches = get_recent_token_launches(days_ago=30, min_liquidity=5000)

# Display recent launches in a table
if recent_launches:
    launch_df = pd.DataFrame(recent_launches)
    
    # Format the display
    display_cols = [
        'token_symbol', 'token_name', 'mint_address', 
        'initial_liquidity_usd', 'initial_mcap_usd', 'launch_date'
    ]
    
    # Convert timestamp to date if needed
    if 'launch_timestamp' in launch_df.columns and 'launch_date' not in launch_df.columns:
        launch_df['launch_date'] = pd.to_datetime(launch_df['launch_timestamp'], unit='s')
    
    # Select columns that exist
    available_cols = [col for col in display_cols if col in launch_df.columns]
    display(launch_df[available_cols].head(10))
    
    # Select tokens for detailed analysis
    selected_tokens = launch_df[['token_symbol', 'mint_address']].head(5).values.tolist()
    print(f"\nSelected {len(selected_tokens)} tokens for detailed analysis")
    for symbol, address in selected_tokens:
        print(f"- {symbol}: {address}")
else:
    print("No recent token launches found, using predefined examples for testing")
    # Use real token examples for testing
    selected_tokens = [
        ["PYTH", "HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB"],  # Pyth token
        ["JTO", "jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY"],     # Jito token
        ["CROWN", "FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2"]  # Crown token
    ]

Searching for token launches in the last 30 days...
Error fetching token launches: 'RugCheckCollector' object has no attribute 'get_recent_token_launches'
No recent token launches found, using predefined examples for testing


## Analyze Token Distribution and Initial Allocation

Let's analyze how the tokens were initially distributed, which is crucial for identifying potential rug pulls:

In [16]:
# Function to analyze token distribution
def analyze_token_distribution(mint_address, token_symbol):
    print(f"\nAnalyzing token distribution for {token_symbol} ({mint_address})")
    
    # Get token holders
    try:
        holders_data = vybe.get_token_top_holders(mint_address)
        holders = holders_data.get("data", [])
        print(f"Found {len(holders)} token holders")
    except Exception as e:
        print(f"Error getting token holders: {e}")
        holders = []
        
    if not holders:
        return None
    
    # Convert to DataFrame
    holders_df = pd.DataFrame(holders)
    
    # Calculate distribution metrics
    total_supply = holders_df["amount"].sum() if "amount" in holders_df.columns else 0
    
    # Calculate concentration metrics
    if "amount" in holders_df.columns and total_supply > 0:
        holders_df["percentage"] = holders_df["amount"] / total_supply * 100
        
        # Top holder percentages
        top1_pct = holders_df["percentage"].iloc[0] if len(holders_df) >= 1 else 0
        top5_pct = holders_df["percentage"].iloc[:5].sum() if len(holders_df) >= 5 else holders_df["percentage"].sum()
        top10_pct = holders_df["percentage"].iloc[:10].sum() if len(holders_df) >= 10 else holders_df["percentage"].sum()
        
        print(f"Top holder owns {top1_pct:.2f}% of supply")
        print(f"Top 5 holders own {top5_pct:.2f}% of supply")
        print(f"Top 10 holders own {top10_pct:.2f}% of supply")
        
        # Calculate Gini coefficient (measure of inequality, 0 = perfect equality, 1 = perfect inequality)
        if len(holders_df) > 5:
            gini = calculate_gini_coefficient(holders_df["amount"].values)
            print(f"Gini coefficient: {gini:.4f}")
        else:
            gini = None
        
        # Identify potential team wallets
        team_wallets = []
        for index, row in holders_df.head(10).iterrows():
            address = row.get("address")
            if address:
                # Check if this is a team wallet using on-chain indicators
                try:
                    is_team = rugcheck.check_if_team_wallet(address, mint_address)
                    if is_team:
                        team_wallets.append({
                            "address": address,
                            "percentage": row.get("percentage", 0),
                            "confidence": is_team.get("confidence", 0)
                        })
                except Exception as e:
                    pass
        
        if team_wallets:
            team_pct = sum(w["percentage"] for w in team_wallets)
            print(f"Identified {len(team_wallets)} potential team wallets holding {team_pct:.2f}% of supply")
        
        # Plot distribution
        plt.figure(figsize=(10, 6))
        plt.pie(holders_df["percentage"].head(10), labels=holders_df["address"].head(10).apply(lambda x: x[:6] + "..."),
                autopct='%1.1f%%', startangle=90)
        plt.title(f"{token_symbol} Token Distribution (Top 10 Holders)")
        plt.axis('equal')
        plt.tight_layout()
        plt.show()
        
        return {
            "token_symbol": token_symbol,
            "mint_address": mint_address,
            "holders_count": len(holders_df),
            "top1_pct": top1_pct,
            "top5_pct": top5_pct,
            "top10_pct": top10_pct,
            "gini_coefficient": gini,
            "team_wallets": team_wallets,
            "team_wallets_pct": sum(w["percentage"] for w in team_wallets) if team_wallets else 0
        }
    else:
        print("Insufficient data for distribution analysis")
        return None

# Analyze distribution for each selected token
distribution_results = {}
for token_symbol, mint_address in selected_tokens:
    distribution_results[mint_address] = analyze_token_distribution(mint_address, token_symbol)

2025-04-25 00:45:26,646 - vybe_collector - INFO - Getting top holders for HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB



Analyzing token distribution for PYTH (HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB)


2025-04-25 00:45:37,746 - vybe_collector - ERROR - Failed to make API request: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB/top-holders?limit=100&page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A7B90>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))
2025-04-25 00:45:37,746 - vybe_collector - INFO - Getting top holders for jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY
2025-04-25 00:45:37,746 - vybe_collector - INFO - Getting top holders for jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY
2025-04-25 00:45:37,790 - vybe_collector - ERROR - Failed to make API request: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY/top-holders?limit=100&page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A5B50>: Failed 

Error getting token holders: Request failed: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB/top-holders?limit=100&page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A7B90>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))

Analyzing token distribution for JTO (jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY)
Error getting token holders: Request failed: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY/top-holders?limit=100&page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A5B50>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))

Analyzing token distribution for CROWN (FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2)


2025-04-25 00:45:38,082 - vybe_collector - ERROR - Failed to make API request: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2/top-holders?limit=100&page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A6840>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))


Error getting token holders: Request failed: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2/top-holders?limit=100&page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A6840>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))


## Analyze Launch Metrics and Price Action

Let's examine the launch metrics and initial price action:

In [17]:
# Function to analyze ICO launch metrics
def analyze_launch_metrics(mint_address, token_symbol):
    print(f"\nAnalyzing launch metrics for {token_symbol} ({mint_address})")
    
    # Get token details
    try:
        token_details = vybe.get_token_details(mint_address)
        print(f"Retrieved token details for {token_details.get('name', 'Unknown')} ({token_details.get('symbol', '???')})")
    except Exception as e:
        print(f"Error getting token details: {e}")
        token_details = {"symbol": token_symbol}
    
    # Get token creator info from RugCheck
    try:
        token_report = rugcheck.get_token_report(mint_address)
        creator_address = token_report.get("creator")
        if creator_address:
            print(f"Creator address: {creator_address}")
    except Exception as e:
        print(f"Error getting token report: {e}")
        token_report = {}
        creator_address = None
    
    # Get token price history to analyze launch
    try:
        # Get data for the first week after launch
        launch_date = token_report.get("createdAt")
        if launch_date:
            launch_timestamp = int(datetime.strptime(launch_date, "%Y-%m-%dT%H:%M:%S.%fZ").timestamp())
            end_timestamp = launch_timestamp + (7 * 24 * 60 * 60)  # 7 days after launch
        else:
            # If no launch date, use the last 7 days
            end_timestamp = int(datetime.now().timestamp())
            launch_timestamp = end_timestamp - (7 * 24 * 60 * 60)
        
        price_data = vybe.get_token_price_ohlcv(
            mint_address, 
            resolution="1h", 
            time_start=launch_timestamp,
            time_end=end_timestamp
        )
        price_history = price_data.get("data", [])
        print(f"Retrieved {len(price_history)} price data points")
    except Exception as e:
        print(f"Error getting price history: {e}")
        price_history = []
    
    # Get launch liquidity data
    try:
        liquidity_data = rugcheck.get_token_liquidity_history(mint_address)
        print(f"Retrieved liquidity history with {len(liquidity_data)} data points")
    except Exception as e:
        print(f"Error getting liquidity data: {e}")
        liquidity_data = []
    
    # Calculate launch metrics
    if price_history:
        launch_metrics = calculate_ico_metrics(price_history, liquidity_data)
        
        # Display launch metrics
        print("\nLaunch Metrics:")
        if "initial_price" in launch_metrics:
            print(f"- Initial price: ${launch_metrics['initial_price']:.6f}")
        if "price_peak" in launch_metrics:
            print(f"- Peak price: ${launch_metrics['price_peak']:.6f}")
        if "price_7d" in launch_metrics:
            print(f"- 7-day price: ${launch_metrics['price_7d']:.6f}")
        if "price_change_pct" in launch_metrics:
            print(f"- 7-day price change: {launch_metrics['price_change_pct']:.2f}%")
        if "initial_liquidity" in launch_metrics:
            print(f"- Initial liquidity: ${launch_metrics['initial_liquidity']:.2f}")
        if "liquidity_change_pct" in launch_metrics:
            print(f"- 7-day liquidity change: {launch_metrics['liquidity_change_pct']:.2f}%")
        
        # Plot price action
        price_df = pd.DataFrame(price_history)
        if not price_df.empty and "timestamp" in price_df and "close" in price_df:
            price_df["datetime"] = pd.to_datetime(price_df["timestamp"], unit="s")
            
            plt.figure(figsize=(12, 6))
            plt.plot(price_df["datetime"], price_df["close"], 'b-')
            plt.title(f"{token_details.get('symbol', token_symbol)} Price Since Launch")
            plt.xlabel("Date")
            plt.ylabel("Price (USD)")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            # Volume chart
            if "volume" in price_df:
                plt.figure(figsize=(12, 4))
                plt.bar(price_df["datetime"], price_df["volume"], color='g', alpha=0.7)
                plt.title(f"{token_details.get('symbol', token_symbol)} Trading Volume Since Launch")
                plt.xlabel("Date")
                plt.ylabel("Volume (USD)")
                plt.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.show()
        
        return {
            "token_symbol": token_symbol,
            "mint_address": mint_address,
            "creator_address": creator_address,
            "launch_metrics": launch_metrics,
            "price_history": price_df.to_dict('records') if not price_df.empty else []
        }
    else:
        print("Insufficient data for launch analysis")
        return {
            "token_symbol": token_symbol,
            "mint_address": mint_address,
            "creator_address": creator_address
        }

# Analyze launch metrics for each selected token
launch_results = {}
for token_symbol, mint_address in selected_tokens:
    launch_results[mint_address] = analyze_launch_metrics(mint_address, token_symbol)

2025-04-25 00:45:38,103 - vybe_collector - INFO - Getting token details for HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB



Analyzing launch metrics for PYTH (HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB)


2025-04-25 00:45:38,417 - vybe_collector - ERROR - Failed to make API request: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A4320>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))
2025-04-25 00:45:38,417 - rugcheck_collector - INFO - Getting token report for HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB
2025-04-25 00:45:38,417 - rugcheck_collector - INFO - Getting token report for HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB


Error getting token details: Request failed: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A4320>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))


2025-04-25 00:45:38,673 - rugcheck_collector - ERROR - Failed to make API request: 401 Client Error: Unauthorized for url: https://api.rugcheck.xyz/v1/tokens/HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB/report
2025-04-25 00:45:38,675 - vybe_collector - INFO - Getting price OHLCV for HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB
2025-04-25 00:45:38,675 - vybe_collector - INFO - Getting price OHLCV for HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB
2025-04-25 00:45:38,751 - vybe_collector - ERROR - Failed to make API request: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/price/HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB/token-ohlcv?limit=100&page=1&resolution=1h&timeStart=1744926338&timeEnd=1745531138 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A74D0>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))
2025-04-25 00:45:38,751 - vybe_collector - INFO - Getting token details for jtojtome

...

Error getting price history: Request failed: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/price/jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY/token-ohlcv?limit=100&page=1&resolution=1h&timeStart=1744926339&timeEnd=1745531139 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0B3C20>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))
Error getting liquidity data: 'RugCheckCollector' object has no attribute 'get_token_liquidity_history'
Insufficient data for launch analysis

Analyzing launch metrics for CROWN (FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2)


2025-04-25 00:45:39,754 - vybe_collector - ERROR - Failed to make API request: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0B0AD0>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))
2025-04-25 00:45:39,754 - rugcheck_collector - INFO - Getting token report for FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2
2025-04-25 00:45:39,754 - rugcheck_collector - INFO - Getting token report for FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2
2025-04-25 00:45:39,912 - rugcheck_collector - ERROR - Failed to make API request: 401 Client Error: Unauthorized for url: https://api.rugcheck.xyz/v1/tokens/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2/report
2025-04-25 00:45:39,913 - vybe_collector - INFO - Getting price OHLCV for FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2
2025-04-25 00:45:39,912 - rugcheck_col

Error getting token details: Request failed: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/token/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0B0AD0>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))
Error getting token report: Request failed: 401 Client Error: Unauthorized for url: https://api.rugcheck.xyz/v1/tokens/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2/report


2025-04-25 00:45:40,087 - vybe_collector - ERROR - Failed to make API request: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/price/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2/token-ohlcv?limit=100&page=1&resolution=1h&timeStart=1744926339&timeEnd=1745531139 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A6360>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))


Error getting price history: Request failed: HTTPSConnectionPool(host='api.vybe.io', port=443): Max retries exceeded with url: /v1/price/FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2/token-ohlcv?limit=100&page=1&resolution=1h&timeStart=1744926339&timeEnd=1745531139 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000029DED0A6360>: Failed to resolve 'api.vybe.io' ([Errno 11001] getaddrinfo failed)"))
Error getting liquidity data: 'RugCheckCollector' object has no attribute 'get_token_liquidity_history'
Insufficient data for launch analysis


## Analyze Team Wallet Activity

Let's analyze the activity of team wallets to detect potential red flags:

In [18]:
# Function to analyze team wallet activity
def analyze_team_wallets(mint_address, token_symbol):
    print(f"\nAnalyzing team wallets for {token_symbol} ({mint_address})")
    
    # Get creator address from launch results
    creator_address = launch_results.get(mint_address, {}).get("creator_address")
    if not creator_address:
        print("Creator address not available")
    else:
        print(f"Analyzing activity of creator address: {creator_address}")
    
    # Get team wallets from distribution results
    team_wallets = distribution_results.get(mint_address, {})
    team_wallets = team_wallets.get("team_wallets", []) if team_wallets else []
    if not team_wallets:
        print("No team wallets identified from distribution analysis")
    else:
        print(f"Found {len(team_wallets)} team wallets for analysis")
    
    # Combine creator address and team wallets for analysis
    analysis_addresses = [w["address"] for w in team_wallets]
    if creator_address and creator_address not in analysis_addresses:
        analysis_addresses.append(creator_address)
    
    if not analysis_addresses:
        print("No addresses available for team wallet analysis")
        return None
    
    team_activity = {}
    for address in analysis_addresses:
        print(f"\nAnalyzing activity of team wallet: {address}")
        
        # Get transaction history
        try:
            tx_history = helius.fetch_transaction_history(address, limit=100)
            print(f"Fetched {len(tx_history)} transactions for team wallet")
        except Exception as e:
            print(f"Error fetching transaction history: {e}")
            tx_history = []
        
        # Get token transfers specific to the project token
        try:
            token_transfers = helius.analyze_token_transfers(address, limit=100)
            # Filter for the specific token we're analyzing
            token_transfers = [tx for tx in token_transfers if tx.get("mint") == mint_address]
            print(f"Fetched {len(token_transfers)} token transfers related to project token")
        except Exception as e:
            print(f"Error analyzing token transfers: {e}")
            token_transfers = []
        
        # Analyze team wallet behavior
        if token_transfers:
            # Calculate total outflows/inflows of project tokens
            outflows = [tx for tx in token_transfers if tx.get("sender_address") == address]
            inflows = [tx for tx in token_transfers if tx.get("receiver_address") == address]
            
            total_outflow = sum(tx.get("amount", 0) for tx in outflows)
            total_inflow = sum(tx.get("amount", 0) for tx in inflows)
            net_flow = total_inflow - total_outflow
            
            print(f"Total token outflow: {total_outflow:,.2f}")
            print(f"Total token inflow: {total_inflow:,.2f}")
            print(f"Net token flow: {net_flow:,.2f}")
            
            # Identify large outflows
            large_outflows = [tx for tx in outflows if tx.get("amount", 0) > (total_inflow * 0.1)]
            if large_outflows:
                print(f"Found {len(large_outflows)} large token outflows (>10% of inflows)")
            
            # Calculate selling pressure
            selling_pressure = total_outflow / total_inflow if total_inflow > 0 else 0
            print(f"Selling pressure: {selling_pressure:.2f} (>0.5 may indicate dumping)")
            
            # Record analysis results
            team_activity[address] = {
                "transaction_count": len(tx_history),
                "token_transfers_count": len(token_transfers),
                "total_outflow": total_outflow,
                "total_inflow": total_inflow,
                "net_flow": net_flow,
                "selling_pressure": selling_pressure,
                "large_outflows": len(large_outflows),
                "is_dumping": selling_pressure > 0.5 and net_flow < 0
            }
        else:
            print("No token transfers found for analysis")
            team_activity[address] = {
                "transaction_count": len(tx_history),
                "token_transfers_count": 0
            }
    
    # Summarize team activity
    dumping_wallets = [addr for addr, data in team_activity.items() if data.get("is_dumping", False)]
    if dumping_wallets:
        print(f"\n⚠️ Warning: {len(dumping_wallets)} team wallets show signs of dumping tokens")
        for addr in dumping_wallets:
            print(f"- {addr} (selling pressure: {team_activity[addr]['selling_pressure']:.2f})")
    else:
        print("\n✅ No team wallets show signs of dumping tokens")
    
    return {
        "token_symbol": token_symbol,
        "mint_address": mint_address,
        "team_wallets": analysis_addresses,
        "team_activity": team_activity,
        "dumping_wallets": dumping_wallets,
        "dumping_detected": len(dumping_wallets) > 0
    }

# Analyze team wallet activity for each token
team_results = {}
for token_symbol, mint_address in selected_tokens:
    team_results[mint_address] = analyze_team_wallets(mint_address, token_symbol)


Analyzing team wallets for PYTH (HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB)
Creator address not available
No team wallets identified from distribution analysis
No addresses available for team wallet analysis

Analyzing team wallets for JTO (jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY)
Creator address not available
No team wallets identified from distribution analysis
No addresses available for team wallet analysis

Analyzing team wallets for CROWN (FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2)
Creator address not available
No team wallets identified from distribution analysis
No addresses available for team wallet analysis


## Calculate ICO Risk Scores

Let's calculate comprehensive risk scores for each ICO based on our analysis:

In [19]:
# Function to calculate ICO risk score
def calculate_ico_risk_score(mint_address, token_symbol):
    print(f"\nCalculating risk score for {token_symbol} ({mint_address})")
    
    # Get data from previous analyses
    distribution_data = distribution_results.get(mint_address, {})
    launch_data = launch_results.get(mint_address, {})
    team_data = team_results.get(mint_address, {})
    
    # Initialize risk score components
    risk_factors = []
    total_score = 0
    max_score = 0
    
    # 1. Distribution risk factors (max 30 points)
    if distribution_data:
        max_score += 30
        
        # Top holder concentration risk (max 15 points)
        top1_pct = distribution_data.get("top1_pct", 0)
        if top1_pct > 50:  # Very high risk if >50%
            concentration_score = 15
        elif top1_pct > 30:  # High risk if >30%
            concentration_score = 10
        elif top1_pct > 15:  # Medium risk if >15%
            concentration_score = 5
        else:  # Low risk
            concentration_score = 0
        
        total_score += concentration_score
        risk_factors.append({
            "name": "Token concentration",
            "description": f"Top holder owns {top1_pct:.1f}% of supply",
            "score": concentration_score,
            "max_score": 15
        })
        
        # Team wallet holdings risk (max 15 points)
        team_pct = distribution_data.get("team_wallets_pct", 0)
        if team_pct > 40:  # Very high risk if >40%
            team_holding_score = 15
        elif team_pct > 25:  # High risk if >25%
            team_holding_score = 10
        elif team_pct > 15:  # Medium risk if >15%
            team_holding_score = 5
        else:  # Low risk
            team_holding_score = 0
        
        total_score += team_holding_score
        if team_pct > 0:
            risk_factors.append({
                "name": "Team holdings",
                "description": f"Team wallets hold {team_pct:.1f}% of supply",
                "score": team_holding_score,
                "max_score": 15
            })
    
    # 2. Launch metrics risk factors (max 40 points)
    if launch_data and "launch_metrics" in launch_data:
        max_score += 40
        launch_metrics = launch_data["launch_metrics"]
        
        # Initial liquidity risk (max 20 points)
        initial_liquidity = launch_metrics.get("initial_liquidity", 0)
        if initial_liquidity < 1000:  # Very high risk if <$1k
            liquidity_score = 20
        elif initial_liquidity < 5000:  # High risk if <$5k
            liquidity_score = 15
        elif initial_liquidity < 20000:  # Medium risk if <$20k
            liquidity_score = 10
        elif initial_liquidity < 50000:  # Low-medium risk if <$50k
            liquidity_score = 5
        else:  # Low risk if >$50k
            liquidity_score = 0
        
        total_score += liquidity_score
        risk_factors.append({
            "name": "Initial liquidity",
            "description": f"${initial_liquidity:,.2f} initial liquidity",
            "score": liquidity_score,
            "max_score": 20
        })
        
        # Price volatility risk (max 20 points)
        price_change = launch_metrics.get("price_change_pct", 0)
        if price_change < -50:  # Very high risk if dropped >50%
            price_score = 20
        elif price_change < -25:  # High risk if dropped >25%
            price_score = 15
        elif price_change > 1000:  # Medium-high risk if pumped >1000% (potential pump & dump)
            price_score = 10
        elif price_change < 0:  # Medium risk if any price drop
            price_score = 5
        else:  # Low risk
            price_score = 0
        
        total_score += price_score
        risk_factors.append({
            "name": "Price action",
            "description": f"{price_change:.1f}% price change in first week",
            "score": price_score,
            "max_score": 20
        })
    
    # 3. Team activity risk factors (max 30 points)
    if team_data:
        max_score += 30
        
        # Team token dumping risk (max 30 points)
        dumping_detected = team_data.get("dumping_detected", False)
        dumping_wallets = team_data.get("dumping_wallets", [])
        team_activity = team_data.get("team_activity", {})
        
        # Calculate average selling pressure
        selling_pressures = [data.get("selling_pressure", 0) for addr, data in team_activity.items() 
                           if "selling_pressure" in data]
        avg_selling_pressure = sum(selling_pressures) / len(selling_pressures) if selling_pressures else 0
        
        if dumping_detected and avg_selling_pressure > 0.8:  # Very high risk
            dumping_score = 30
        elif dumping_detected and avg_selling_pressure > 0.5:  # High risk
            dumping_score = 20
        elif avg_selling_pressure > 0.3:  # Medium risk
            dumping_score = 10
        else:  # Low risk
            dumping_score = 0
        
        total_score += dumping_score
        risk_description = f"Team selling pressure: {avg_selling_pressure:.2f}"
        if dumping_detected:
            risk_description += f" ({len(dumping_wallets)} wallets dumping)"
        
        risk_factors.append({
            "name": "Team token selling",
            "description": risk_description,
            "score": dumping_score,
            "max_score": 30
        })
    
    # Calculate final risk percentage (0-100%)
    risk_percentage = (total_score / max_score * 100) if max_score > 0 else 0
    
    # Determine risk level
    if risk_percentage >= 75:
        risk_level = "Very High Risk"
    elif risk_percentage >= 50:
        risk_level = "High Risk"
    elif risk_percentage >= 25:
        risk_level = "Medium Risk"
    else:
        risk_level = "Low Risk"
    
    print(f"Risk score: {risk_percentage:.1f}% ({risk_level})")
    print("Top risk factors:")
    for factor in sorted(risk_factors, key=lambda x: x["score"], reverse=True)[:3]:  # Top 3 factors
        print(f"- {factor['name']}: {factor['description']} (score: {factor['score']}/{factor['max_score']})")
    
    return {
        "token_symbol": token_symbol,
        "mint_address": mint_address,
        "risk_score": risk_percentage,
        "risk_level": risk_level,
        "risk_factors": risk_factors
    }

# Calculate risk scores for each token
risk_results = {}
for token_symbol, mint_address in selected_tokens:
    risk_results[mint_address] = calculate_ico_risk_score(mint_address, token_symbol)


Calculating risk score for PYTH (HZ1JovNiVvGrGNiiYvEozEVgZ58xaU3HFnNnfmj8UrAB)
Risk score: 0.0% (Low Risk)
Top risk factors:

Calculating risk score for JTO (jtojtomepa8beP8AuQc6eXt5FriJwTMt8qPucZNEBxY)
Risk score: 0.0% (Low Risk)
Top risk factors:

Calculating risk score for CROWN (FfmxuKM3dxhJuosGn1S4KKgVGTxj5FcVWHf9Rs4zC5K2)
Risk score: 0.0% (Low Risk)
Top risk factors:


## Conclusions and Report Generation

Let's summarize our findings and generate a comprehensive ICO analysis report:

In [20]:
# Generate ICO analysis report
def generate_ico_report():
    report = ["# Solana ICO Analysis Report\n"]
    report.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    
    # Summary table of all analyzed tokens
    report.append("## Summary of Analyzed Tokens\n")
    report.append("| Token | Risk Level | Risk Score | Initial Liquidity | Top Holder % | Team Dumping |")
    report.append("| ----- | ---------- | ---------- | ----------------- | ------------ | ------------ |")
    
    for token_symbol, mint_address in selected_tokens:
        # Get data for token
        risk_data = risk_results.get(mint_address, {})
        distribution_data = distribution_results.get(mint_address, {}) or {}
        launch_data = launch_results.get(mint_address, {}) or {}
        launch_metrics = launch_data.get("launch_metrics", {}) or {}
        team_data = team_results.get(mint_address, {}) or {}
        
        # Format table row
        risk_level = risk_data.get("risk_level", "Unknown") if risk_data else "Unknown"
        risk_score = f"{risk_data.get('risk_score', 0):.1f}%" if risk_data and "risk_score" in risk_data else "N/A"
        liquidity = f"${launch_metrics.get('initial_liquidity', 0):,.2f}" if launch_metrics and "initial_liquidity" in launch_metrics else "N/A"
        top_holder = f"{distribution_data.get('top1_pct', 0):.1f}%" if distribution_data and "top1_pct" in distribution_data else "N/A"
        team_dumping = "Yes" if team_data and team_data.get("dumping_detected", False) else "No"
        
        report.append(f"| {token_symbol} | {risk_level} | {risk_score} | {liquidity} | {top_holder} | {team_dumping} |")
    
    # Detailed analysis for each token
    for token_symbol, mint_address in selected_tokens:
        report.append(f"\n## {token_symbol} ({mint_address[:8]}...)\n")
        
        # Risk assessment
        risk_data = risk_results.get(mint_address, {})
        if risk_data is not None and isinstance(risk_data, dict):
            report.append(f"### Risk Assessment\n")
            report.append(f"- **Overall risk score**: {risk_data.get('risk_score', 0):.1f}%")
            report.append(f"- **Risk level**: {risk_data.get('risk_level', 'Unknown')}\n")
            
            report.append("#### Risk Factors\n")
            risk_factors = risk_data.get("risk_factors", [])
            if risk_factors:
                for factor in sorted(risk_factors, key=lambda x: x["score"], reverse=True):
                    report.append(f"- **{factor['name']}**: {factor['description']} (score: {factor['score']}/{factor['max_score']})")
            else:
                report.append("- No specific risk factors identified")
        
        # Token distribution
        distribution_data = distribution_results.get(mint_address, {})
        if distribution_data is not None and isinstance(distribution_data, dict):
            report.append(f"\n### Token Distribution\n")
            report.append(f"- Total holders: {distribution_data.get('holders_count', 'Unknown')}")
            report.append(f"- Top holder percentage: {distribution_data.get('top1_pct', 0):.1f}%")
            report.append(f"- Top 5 holders percentage: {distribution_data.get('top5_pct', 0):.1f}%")
            report.append(f"- Top 10 holders percentage: {distribution_data.get('top10_pct', 0):.1f}%")
            
            if "gini_coefficient" in distribution_data and distribution_data["gini_coefficient"] is not None:
                report.append(f"- Gini coefficient: {distribution_data['gini_coefficient']:.4f} (0=equal distribution, 1=completely unequal)")
            
            team_wallets = distribution_data.get("team_wallets", [])
            if team_wallets:
                report.append(f"- Team wallets control: {distribution_data.get('team_wallets_pct', 0):.1f}% of supply")
                report.append("\n#### Identified Team Wallets\n")
                for wallet in team_wallets:
                    report.append(f"- `{wallet['address']}`: {wallet['percentage']:.1f}% of supply (confidence: {wallet['confidence']:.2f})")
        else:
            report.append(f"\n### Token Distribution\n")
            report.append("- Distribution data not available")
        
        # Launch metrics
        launch_data = launch_results.get(mint_address, {})
        launch_metrics = launch_data.get("launch_metrics", {}) if launch_data else {}
        if launch_metrics is not None and isinstance(launch_metrics, dict):
            report.append(f"\n### Launch Metrics\n")
            if "initial_price" in launch_metrics:
                report.append(f"- Initial price: ${launch_metrics['initial_price']:.6f}")
            if "price_peak" in launch_metrics:
                report.append(f"- Peak price: ${launch_metrics['price_peak']:.6f}")
            if "price_7d" in launch_metrics:
                report.append(f"- 7-day price: ${launch_metrics['price_7d']:.6f}")
            if "price_change_pct" in launch_metrics:
                report.append(f"- 7-day price change: {launch_metrics['price_change_pct']:.2f}%")
            if "initial_liquidity" in launch_metrics:
                report.append(f"- Initial liquidity: ${launch_metrics['initial_liquidity']:.2f}")
            if "liquidity_change_pct" in launch_metrics:
                report.append(f"- 7-day liquidity change: {launch_metrics['liquidity_change_pct']:.2f}%")
        else:
            report.append(f"\n### Launch Metrics\n")
            report.append("- Launch metrics data not available")
        
        # Team wallet activity
        team_data = team_results.get(mint_address, {})
        if team_data is not None and isinstance(team_data, dict):
            report.append(f"\n### Team Wallet Activity\n")
            
            if team_data.get("dumping_detected", False):
                report.append(f"⚠️ **Warning: Team wallets showing signs of dumping tokens**\n")
            else:
                report.append(f"✅ No evidence of team dumping detected\n")
            
            team_activity = team_data.get("team_activity", {})
            if team_activity:
                for addr, activity in team_activity.items():
                    if isinstance(activity, dict) and "selling_pressure" in activity:
                        status = "🔴 Dumping" if activity.get("is_dumping", False) else "🟢 Normal"
                        report.append(f"- `{addr[:8]}...`: {status} (selling pressure: {activity['selling_pressure']:.2f}, net flow: {activity['net_flow']:,.0f} tokens)")
        else:
            report.append(f"\n### Team Wallet Activity\n")
            report.append("- Team wallet activity data not available")
    
    # General recommendations
    report.append("\n## General Recommendations for ICO Investors\n")
    report.append("1. **Check token distribution** - Be cautious of tokens where top holders control >50% of supply")
    report.append("2. **Verify liquidity** - Higher liquidity (>$50k) reduces manipulation risk")
    report.append("3. **Monitor team wallet activity** - Watch for team wallets selling large amounts early")
    report.append("4. **Examine price action** - Extreme volatility may indicate market manipulation")
    report.append("5. **Look for locked liquidity** - Tokens with locked liquidity are generally safer")
    report.append("6. **Diversify investments** - Never put all your funds into a single ICO")
    
    return "\n".join(report)

# Generate and save the report
report_content = generate_ico_report()

# Save report to file
report_path = f"../../reports/ico_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
os.makedirs("../../reports", exist_ok=True)

with open(report_path, "w") as f:
    f.write(report_content)

print(f"\nReport generated and saved to {report_path}")

# Display report in notebook
from IPython.display import Markdown
display(Markdown(report_content))


Report generated and saved to ../../reports/ico_analysis_report_20250425_004540.md


# Solana ICO Analysis Report

Generated on: 2025-04-25 00:45:40

## Summary of Analyzed Tokens

| Token | Risk Level | Risk Score | Initial Liquidity | Top Holder % | Team Dumping |
| ----- | ---------- | ---------- | ----------------- | ------------ | ------------ |
| PYTH | Low Risk | 0.0% | N/A | N/A | No |
| JTO | Low Risk | 0.0% | N/A | N/A | No |
| CROWN | Low Risk | 0.0% | N/A | N/A | No |

## PYTH (HZ1JovNi...)

### Risk Assessment

- **Overall risk score**: 0.0%
- **Risk level**: Low Risk

#### Risk Factors

- No specific risk factors identified

### Token Distribution

- Distribution data not available

### Launch Metrics


### Team Wallet Activity

- Team wallet activity data not available

## JTO (jtojtome...)

### Risk Assessment

- **Overall risk score**: 0.0%
- **Risk level**: Low Risk

#### Risk Factors

- No specific risk factors identified

### Token Distribution

- Distribution data not available

### Launch Metrics


### Team Wallet Activity

- Team wallet activity data not available

## CROWN (FfmxuKM3...)

### Risk Assessment

- **Overall risk score**: 0.0%
- **Risk level**: Low Risk

#### Risk Factors

- No specific risk factors identified

### Token Distribution

- Distribution data not available

### Launch Metrics


### Team Wallet Activity

- Team wallet activity data not available

## General Recommendations for ICO Investors

1. **Check token distribution** - Be cautious of tokens where top holders control >50% of supply
2. **Verify liquidity** - Higher liquidity (>$50k) reduces manipulation risk
3. **Monitor team wallet activity** - Watch for team wallets selling large amounts early
4. **Examine price action** - Extreme volatility may indicate market manipulation
5. **Look for locked liquidity** - Tokens with locked liquidity are generally safer
6. **Diversify investments** - Never put all your funds into a single ICO